# Non-Parametric-Learners

## kNN and Decision Trees

### Moses Marsh
#### May 24, 2017

(based heavily on the notebooks from Jack Bennetto & Jonathan Torrez)

## Agenda

#### Morning: kNN

 * Non-parametric models
 * Understanding kNN
 * Scaling
 * Distance metrics
 * Curse of Dimensionality
 
#### Afternoon: Decision Trees
 * Entropy and purity
 * Recursion
 * Decision-tree algorithm
 * Gini and regression
 * Pre- and post- pruning

## Morning Objectives [morning-kNN]

* **Implement** pseudocode for kNN algorithm
* **State** common distance metrics used for kNN
* **Describe** effect of varying k (num_neigbors)
* **Explain** importance of scaling for kNN

## Parametric vs. Non-Parametric Models


### Parametric Models

#### Linear Regression


$$\hat y = \hat\beta_0 + \hat\beta_1 X_1 + \hat\beta_2 X_2 + ... + \hat\beta_i X_i $$
    
How many parameters can we describe it with?

Can we ever change the number of parameters?

What does using this model mean about the structure of our data?

**Hint:** alternate way to write linear regression equation?


## Properties of Parametric Models

In [ ]:
# load answers after class discussion
with open('answers/prop_param.txt') as f:
    for line in f:
        print line

#### Examples:
* Linear regression
* Logistic regression

## Properties of Non-Parametric Models

Class discussion: What do you think they are?

In [ ]:
# load answers after class discussion
with open('answers/prop_non_param.txt') as f:
    for line in f:
        print line

Class discussion: contrast the pros and cons of these two types of models

## Pros-Cons of Parametric Models

In [ ]:
# load answers after class discussion
with open('answers/pc_param.txt') as f:
    for line in f:
        print line

## Pros-Cons of Non-parametric Models

In [ ]:
# load answers after class discussion
with open('answers/pc_nonparam.txt') as f:
    for line in f:
        print line

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as scs
import operator
import math
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets, tree
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

%matplotlib inline
# Make it pretty
plt.style.use('ggplot')

# Seed random functions for reproducibility
#np.random.seed(12345)

In [ ]:
def make_horse_dog_data(n_horses, n_dogs, messiness=1):
    """Return sample of horse dog data
    
    Parameters
    ----------
    n_horses: int
        The number of horses
    n_dogs: numpy array
        The number of dogs
    messiness: float
        How much to multiple by the standard deviations of the logs (default 1)
        
    Returns
    -------
    horse_weight: numpy array
        The weights of the horses
    horse_height: numpy array
        The height of the horses
    dog_weight: numpy array
        The weight of the dogs
    dog_height: numpy array
        The height of the dogs
    """

    horse_weight = scs.norm(3,0.15*messiness).rvs(n_horses)
    horse_height = horse_weight * 0.4 + scs.norm(-0.5,0.05*messiness).rvs(n_horses)
    horse_weight = 10**horse_weight
    horse_height = 10**horse_height
    dog_weight = np.zeros(n_dogs)
    dog_height = np.zeros(n_dogs)

    dog_weight[:n_dogs//2] = scs.norm(1.7,0.20*messiness).rvs(n_dogs//2)
    dog_weight[n_dogs//2:] = scs.norm(2.5,0.25*messiness).rvs(n_dogs//2)
    dog_height[:n_dogs//2] = dog_weight[:n_dogs//2]*0.50 + scs.norm(-0.5,0.2*messiness).rvs(n_dogs//2)
    dog_height[n_dogs//2:] = dog_weight[n_dogs//2:]*0.30 + scs.norm(-0.5,0.1*messiness).rvs(n_dogs//2)
    dog_weight = 10**dog_weight
    dog_height = 10**dog_height
    return horse_weight, horse_height, dog_weight, dog_height

In [ ]:
# Make some fake data

horse_weight, horse_height, dog_weight, dog_height = make_horse_dog_data(100, 40)

In [ ]:
def plot_horse_dog_data(ax, horse_weight, horse_height, dog_weight, dog_height):
    """Plot the horse dog data"""
    ax.plot(horse_weight, horse_height, 'ro', label='Horse')
    ax.plot(dog_weight, dog_height, 'bo', label='Dog')
    ax.set_title("Horse vs Dog")
    ax.set_xlabel("Weight (lbs)")
    ax.set_ylabel("Height (ft)")
    ax.legend(loc='upper left')


In [ ]:
fig, ax = plt.subplots()
plot_horse_dog_data(ax, horse_weight, horse_height, dog_weight, dog_height)

## What if we get new data? 

In [ ]:
new_weights = [200, 500, 1200]
new_heights = [0.7, 2.5, 5.7]

## Are they horses or dogs?

In [ ]:
# Plot the new points

fig, ax = plt.subplots()

ax.plot(new_weights, new_heights, marker='o', markersize=10, color="yellow", linestyle='None')


# Plot the original data
plot_horse_dog_data(ax, horse_weight, horse_height, dog_weight, dog_height)

# Draw arrows highlighting the new points
for new_x, new_y in zip(new_weights, new_heights):
    ax.annotate("",
                xy=(new_x, new_y), xycoords='data',
                xytext=(100, -7), textcoords='offset points',
                arrowprops=dict(facecolor='black', shrink=0.05))
    

## What algorithm did you use?

Class discussion

## kNN pseudocode

Class discussion: What do you think this is? Describe both the training and prediction phases.

In [ ]:
# load answers after class discussion
with open('answers/knn_pseudocode.txt') as f:
    for line in f:
        print line

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

a = np.array([[2.1, 3.2, 2.0, 2.9, 4.1, 3.0, 4.1, 6.2, 8.4, 4.1, 4.9, 5.8, 8.1, 5.0, 7.2],
              [3.1, 3.2, 4.0, 3.9, 4.0, 5.1, 4.9, 5.0, 5.1, 6.2, 5.9, 6.1, 6.0, 7.2, 6.8]])
b = np.array([[1.2, 1.8, 3.0, 5.1, 6.1, 1.3, 2.9, 3.7, 7.1, 6.2, 6.9, 8.0, 2.7, 0.3, 1.1, 6.9, 8.1, 0.0, 2.1, 1.2, 3.3, 2.1, 2.9],
              [1.1, 1.8, 1.0, 0.9, 1.1, 2.3, 1.9, 2.3, 1.9, 3.1, 3.2, 3.0, 3.7, 4.3, 4.1, 3.9, 4.1, 5.1, 5.2, 6.3, 5.8, 7.1, 6.7]])
center = np.array([[2.5], [3.5]])
ax.plot(a[0], a[1], 'ro')
ax.plot(b[0], b[1], 'bo')
ax.plot(center[0], center[1], 'kx', ms=10) 
#ax.add_artist(plt.Circle(center, 0.4, fc='None', ec='k', lw=0.5))
#ax.add_artist(plt.Circle(center, 1.0, fc='None', ec='k', lw=0.5))
#ax.add_artist(plt.Circle(center, 2.0, fc='None', ec='k', lw=0.5))
ax.set_aspect('equal')
plt.axis('off')
plt.show()

What do we predict for different values of k?

In [ ]:
# Make some data that isn't quite as nice
horse_weight, horse_height, dog_weight, dog_height = make_horse_dog_data(50, 30, messiness=1.5)
fig, ax = plt.subplots()
plot_horse_dog_data(ax, horse_weight, horse_height, dog_weight, dog_height)

In [ ]:
def prep_data_for_model(horse_weight, horse_height, dog_weight, dog_height):
    """Return horse dog data in format for kNN classification.
    
    Parameters
    ----------
    horse_weight: numpy array
        The weights of the horses
    horse_height: numpy array
        The height of the horses
    dog_weight: numpy array
        The weight of the dogs
    dog_height: numpy array
        The height of the dogs
        
    Returns
    -------
    X: numpy array, shape = [n_samples, 2]
        The features (weight, height) of the data
    y: numpy array, shape = [n_samples,]
        The labels for our classification (horse=0, dog=1)
    """
    X = np.column_stack((np.concatenate((horse_weight, dog_weight)),
                         np.concatenate((horse_height, dog_height))))
    y = np.concatenate((np.zeros(horse_weight.shape),
                        np.ones(dog_weight.shape)))
    return X, y

In [ ]:
# Function to make getting new samples for model input easier, laziness always wins
def get_new_ugly_sample():
    """Return new sample ready for modeling.
    
    Combines make_ugly_data and prep_data_for_model, see the docs for those functions
    """
    horse_weight, horse_height, dog_weight, dog_height = make_horse_dog_data(100, 60, messiness=1.5)
    X, y = prep_data_for_model(horse_weight, horse_height, dog_weight, dog_height)
    return X, y

In [ ]:
def plot_mult_decision_boundary(ax, X, y, k, scaled=True, title='Title', xlabel='xlabel', ylabel='ylabel'):
    """Plot the decision boundary of a kNN classifier.
    
    Builds and fits a sklearn kNN classifier internally.

    X must contain only 2 continuous features.

    Function modeled on sci-kit learn example.

    Parameters
    ----------
    ax: Matplotlib axes object
        The plot to draw the data and boundary on
        
    X: numpy array
        Training data
    
    y: numpy array
        Target labels
    
    k: int
        The number of neighbors that get a vote.
        
    scaled: boolean, optional (default=True)
        If true scales the features, else uses features in original units
    
    title: string, optional (default = 'Title')
        A string for the title of the plot
    
    xlabel: string, optional (default = 'xlabel')
        A string for the label on the x-axis of the plot
    
    ylabel: string, optional (default = 'ylabel')
        A string for the label on the y-axis of the plot
    

    Returns
    -------
    None
    """
    x_mesh_step_size = 5.0
    y_mesh_step_size = 0.05
    
    #Hard code in colors for classes, one class in red, one in blue
    cmap_light = ListedColormap(['#FFAAAA', '#AAAAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#0000FF'])
    
    #Build a kNN classifier
    clf = neighbors.KNeighborsClassifier(n_neighbors=k, weights='uniform')
    
    if scaled:
        #Build pipeline to scale features
        clf = make_pipeline(StandardScaler(), clf)
        clf.fit(X, y)
    else:
        clf.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, m_max]x[y_min, y_max].
    x_min, x_max = 0, 3000
    y_min, y_max = 0, 10
    #x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    #y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, x_mesh_step_size),
                         np.arange(y_min, y_max, y_mesh_step_size))
    dec_boundary = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    dec_boundary = dec_boundary.reshape(xx.shape)
    ax.pcolormesh(xx, yy, dec_boundary, cmap=cmap_light)

    # Plot also the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())

    ax.set_title(title + ", k={0}, scaled={1}".format(k, scaled))
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
    ax.set_xlim(xmin=x_min, xmax=x_max)
    ax.set_ylim(ymin=y_min, ymax=y_max)

In [ ]:
# Get a sample to plot
X, y = get_new_ugly_sample()

In [ ]:
# Save some titles and labels to make our plots pretty
title = 'Horse vs. Dog'
xlabel = 'Weight (lbs)'
ylabel = 'Height (ft)'

In [ ]:
some_ks = [1, 3, 5, 10, 25, 50]

# Building my plot, 3 rows x 2 columns
fig, axs = plt.subplots(3, 2, figsize=(14, 14))
 
# Loop through the possible ks, put each decision boundary a separate axes
for k, ax in zip(some_ks, axs.flatten()):
    plot_mult_decision_boundary(ax, X, y, k=k, title=title, xlabel=xlabel, ylabel=ylabel)

# Keep things from overlapping
plt.tight_layout()
# Show me the money
plt.show()

### k start

In general, a good starting point for k is $\sqrt{n}$

## Bias-Variance

Class discussion: How does changing k affect each? Why?

### Bias

In [ ]:
# load answers after class discussion
with open('answers/bias_knn.txt') as f:
    for line in f:
        print line

### Variance

In [ ]:
# load answers after class discussion
with open('answers/variance_knn.txt') as f:
    for line in f:
        print line

In [ ]:
# Take 4 different samples from the same population and save them in a list
samples_from_same_pop = []

for _ in xrange(4):
    sample = get_new_ugly_sample()
    samples_from_same_pop.append(sample)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

for sample, ax in zip(samples_from_same_pop, axs.flatten()):
    X = sample[0]
    y = sample[1]
    plot_mult_decision_boundary(ax, X, y, k=1, title=title, xlabel=xlabel, ylabel=ylabel)

plt.tight_layout()
# Offset the figure title to make it look nice
plt.subplots_adjust(top=0.9)
# Set a title for the entire figure
plt.suptitle('Low Bias, High Variance', fontsize=20, weight='bold')
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

for sample, ax in zip(samples_from_same_pop, axs.flatten()):
    X = sample[0]
    y = sample[1]
    plot_mult_decision_boundary(ax, X, y, k=50, title=title, xlabel=xlabel, ylabel=ylabel)
    
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.suptitle('High Bias, Low Variance', fontsize=20, weight='bold')
plt.show()

## Scaling Could Be Important

So far we've been scaling the features by default. What if we didn't?

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 4))
                        
plot_mult_decision_boundary(axs[0], X, y, k=1, title=title, xlabel=xlabel, ylabel=ylabel)
plot_mult_decision_boundary(axs[1], X, y, k=1, scaled=False, title=title, xlabel=xlabel, ylabel=ylabel)

plt.tight_layout()
plt.show()

Scaling is important for any algorithms that use distance in feature space.  If changing the units will change the predictions, scale the data first.

## Distance Metrics

A few common distance metrics, but there are many more.

### Manhattan Distance
City blocks, L1

*Manhattan* distance is the distance as measured along axes at right angles:

$$\sum_i |a_i - b_i|$$


### Euclidean Distance

Straight line, L2

*Euclidean* distance is the distance metric you've probably heard of before:

$$ d(\mathbf{a}, \mathbf{b}) = ||\mathbf{a} - \mathbf{b}|| \ = \sqrt{\sum (a_i - b_i)^2} $$

### Cosine Similarity

Angle

*Cosine* similarity is another commonly used distance metric. It's measuring the angle between the two vectors:

$$ d(\mathbf{a}, \mathbf{b}) = \frac{\mathbf{a} \cdot \mathbf{b}}{||\mathbf{a}||  ||\mathbf{b}||} $$


#### sklearn default is Minkowski (with p=2 which is really just euclidean, p=1 is manhattan)

## Curse of Dimensionality

### Perspective 1: Sample Density

Sampling density is proportional to $$N^\frac{1}{p}$$

where N is the number of samples and p is the number of dimensions.

Let's consider a data set with 100 samples that all have only one feature/predictor. But, you feel that one feature doesn't tell you enough to properly predict anything, so you set out to collect new data. The new data will have ten features/predictors for each sample. How many samples do we need to have an equally dense sample as our original dataset?

#### Original Sample

$$\begin{align}
          density & = N^\frac{1}{p} \\
          & = 100^\frac{1}{1} \\
          & = 100 
\end{align}$$

#### New Sample

$$\begin{align}
          N & = density^ p \\
          &= 100^{10} \\
          &= 100,000,000,000,000,000,000
\end{align}$$

No big, just one hundred quadrillion samples...

### Perspective 2: Loss of Locality

As we increase dimensionality, we lose the concept of locality and things get infinitely far apart.

More precisely, samples that are similar no longer look similar and "closeness" becomes more arbitrary than meaningful. 

For example, in Euclidean distance we must be close in ***all*** dimensions to be considered close.

Close in 2 of 3 dimensions, but far in the 3rd dimension makes the points far apart.



## Pros and Cons of kNN

Class discussion: what do you think they are? why?

In [ ]:
# load answers after class discussion
with open('answers/pc_knn.txt') as f:
    for line in f:
        print line

## kNN Variants

Here are a few common variants on the kNN algorithm.

### Weighted Voting

<img src="images/weighted_knn.png" width=500 align="left"/>

Let the k nearest points have distances $$d_1, d_2,..., d_k$$

The ith point votes with weight of $$\frac{1}{d_i}$$

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

a = np.array([[2.1, 3.2, 2.0, 2.9, 4.1, 3.0, 4.1, 6.2, 8.4, 4.1, 4.9, 5.8, 8.1, 5.0, 7.2],
              [3.1, 3.2, 4.0, 3.9, 4.0, 5.1, 4.9, 5.0, 5.1, 6.2, 5.9, 6.1, 6.0, 7.2, 6.8]])
b = np.array([[1.2, 1.8, 3.0, 5.1, 6.1, 1.3, 2.9, 3.7, 7.1, 6.2, 6.9, 8.0, 2.7, 0.3, 1.1, 6.9, 8.1, 0.0, 2.1, 1.2, 3.3, 2.1, 2.9],
              [1.1, 1.8, 1.0, 0.9, 1.1, 2.3, 1.9, 2.3, 1.9, 3.1, 3.2, 3.0, 3.7, 4.3, 4.1, 3.9, 4.1, 5.1, 5.2, 6.3, 5.8, 7.1, 6.7]])
center = np.array([[2.5], [3.5]])
asize = 50/((a - center)**2).sum(axis=0)
bsize = 50/((b - center)**2).sum(axis=0)
ax.scatter(a[0], a[1], c='r', s=asize)
ax.scatter(b[0], b[1], c='b', s=bsize)
ax.plot(center[0], center[1], 'kx', ms=10) 
ax.add_artist(plt.Circle(center, 0.4, fc='None', ec='k', lw=0.5))
ax.add_artist(plt.Circle(center, 1.0, fc='None', ec='k', lw=0.5))
ax.add_artist(plt.Circle(center, 2.0, fc='None', ec='k', lw=0.5))
ax.set_aspect('equal')
plt.axis('off')
plt.show()

### Regression kNN

<img src="images/regression_knn.png" width=500 align="left"/>

Let the k nearest points have distances $$d_1, d_2,..., d_k$$

Let the k nearest points have targets $$t_1, t_2,..., t_k$$

Predict the mean value ***or*** predict a weighted average

## Morning Objectives [morning-kNN]
Reinforces the following objectives

* **Implement** pseudocode for kNN algorithm
    * Writing a class that implements the kNN algorithm
* **State** common distance metrics used for kNN
    * Writing code that calculates two most common
* **Describe** effect of varying k (num_neigbors)
    * Cross validating to choose k
    * Plotting decision boundary can visualize the effect
* **Explain** importance of scaling for kNN
    * Plotting decision boundary can visualize the effect

# Afternoon Lecture

## Objectives [afternoon-DT]

* **Describe** pros/cons of decision tree algorithm
* **Implement** pseudocode for decision tree algorithm
* **Describe** common measures for making splits in a decision tree
* **Demonstrate** concept of recursion and **relate** it to decision trees
* **State** pruning techniques and pros/cons of each

In [ ]:
# Read in our data
tennis_df = pd.read_table('data/tennis.txt', delim_whitespace=True)

In [ ]:
# Clean up a few things, based on my preferences and making calculating probabilities easier
tennis_df.rename(columns={'playtennis': 'played'}, inplace=True)
tennis_df['played'] = tennis_df['played'].apply(lambda x: 1 if x == 'yes' else 0)

In [ ]:
# Let's look at the data
tennis_df.sort_values('played')

## How Would You Determine If You Played Tennis?

Class discussion

## Quantifying Our Decisions

Does windiness do a good job of predicting "played"? Let's split our dataset into two subsets: points with `wind == True` and points with `wind == False`

In [ ]:
mask = tennis_df['wind'] == True
print tennis_df[mask]
print '--'*25
print tennis_df[~mask]

How much did this help us? A perfect decision would have separated all the `played == 0` points from the `played == 1` points, and a perfectly useless decision would have given us two subsets of the data that each had equal amounts of `0`s and `1`s. Can we make a metric for how "mixed up" or "impure" the set of labels is?

### Shannon Entropy

One common measure of impurity. Let $X$ be a discrete random variable with probability mass function $P(X)$.

For today's purposes, the values of $X$ are _class labels_, and $P(X=i)=p_i$ is the _probability of drawing a point with class $i$_

The entropy is defined as:
$$\begin{align}
H(X) & = E[log_2(\tfrac{1}{P(X)})] \\
     & = -E[log_2(P(X))] \\
     & = - \sum_i p_i log_2(p_i)
\end{align}
$$

### Entropy Graph of Bernoulli Random Variable

<img src="images/entropy_graph.png" width=500 align="left"/>

In [ ]:
def calc_entropy(y):
    """Return the entropy of the array y.
    
    Parameters
    ----------
    y: 1d numpy array
    
    Returns
    -------
    float
    """
    total_samples = y.shape[0]
    summation = 0
    
    for class_i in np.unique(y):
        prob = sum(y == class_i) / float(total_samples)
        summation += prob * np.log2(prob)
    
    return -summation

In [ ]:
y = tennis_df['played'].values

print "Entropy of original data set is {}".format(calc_entropy(y))

### Information Gain 
Information gain is the difference between the _impurity of the parent node_ and the _weighted impurities of the child nodes_. A higher IG for a split means that the split did a better job of separating (or un-mixing) the class labels. 

$$\text{IG}(S, C) = H(S) - \sum_{C_i \in C} \frac{|C_i|}{|S|} H(C_i)$$


In [ ]:
def calc_info_gain(y, y1, y2, impurity_func):
    """Return the information gain of making the given split.
    
    Parameters
    ----------
    y: 1d numpy array
        Labels for parent node
    
    y1: 1d numpy array
        Labels for potential child node 1
    
    y2: 1d numpy array
        Labels for potential child node 2
    
    impurity_func: function
        Function which calculates the impurity of the node 
        (e.g. Shannon Entropy)
    
    Returns
    -------
    float
    """
    total_samples = float(y.shape[0])
    child_imp = 0
    y_impurity = impurity_func(y)
    
    for child_node in (y1, y2):
        child_num = child_node.shape[0]
        child_imp += (child_num / total_samples) * impurity_func(child_node)
        
    return y_impurity - child_imp

In [ ]:
# Let's split on the temperature.

y1 = tennis_df[tennis_df['temperature'] == 'hot']['played'].values
y2 = tennis_df[tennis_df['temperature'] != 'hot']['played'].values

In [ ]:
print "Information Gain is {0}.".format(calc_info_gain(y, y1, y2, calc_entropy))

In [ ]:
# Not much gain, maybe I really liked playing when the temperature was mild
# Let's try that split

y1 = tennis_df[tennis_df['temperature'] == 'mild']['played'].values
y2 = tennis_df[tennis_df['temperature'] != 'mild']['played'].values

In [ ]:
print "Information Gain is {0}.".format(calc_info_gain(y, y1, y2, calc_entropy))

In [ ]:
# Still not much headway, let's just try every possible split and see what works
# And also confirm what we used our common sense algorithm we started with

possible_splits = {}
# Get just my features from the dataframe
feature_cols = tennis_df.drop('played', axis=1).columns

# For a given column, find all the unique possible values
for col in feature_cols:
    col_splits = np.unique(tennis_df[col])
    # For each possible value, split the dataset using that value
    for pos_val in col_splits:
        y1 = tennis_df[tennis_df[col] == pos_val]['played'].values
        y2 = tennis_df[tennis_df[col] != pos_val]['played'].values
        # Calculate the information gain, save it for later
        ig = calc_info_gain(y, y1, y2, calc_entropy)
        key = "{0}: {1}".format(col, pos_val)
        possible_splits[key] = ig

In [ ]:
# Print out our results in a pretty way
colname1 = "Col Name: Value"
colname2 = "Information Gain"

# :20 is specifying a column width, https://docs.python.org/3/library/string.html#formatspec
print "{0:20} || {1}".format(colname1, colname2)
print "-----------------------------------------"
# operator.itemgetter(1) allows us to sort by the second item of the tuple
for k,v in sorted(possible_splits.items(), key=operator.itemgetter(1), reverse=True):
    print "{0:20} || {1}".format(k, v)
    

In [ ]:
# Splitting on overcast gave us the largest info gain
# Let's look at what those child nodes look like and their respective entropies
child_node_left = tennis_df[tennis_df['outlook'] == 'overcast']
print "Entropy of left child node is {0}".format(
          calc_entropy(child_node_left['played'].values))
child_node_left

In [ ]:
# Ok, we got a pure node, that makes sense.
# What about the other node?
child_node_right = tennis_df[tennis_df['outlook'] != 'overcast']
print "Entropy of right child node is {0}".format(
          calc_entropy(child_node_right['played'].values))
child_node_right

Class discussion: why in the world would we want a node that has maximum entropy?

## How would you know how to stop?

Class discussion: When would you stop? Potential problems?

Will get to this soon.

## Other Ways to Calculate Impurity

### Gini Index

This is the default for sci-kit learn trees.

It is a measure of this probability:

* Take a random element from the set.
* Label it randomly according to the distribution of labels in the set.
* What is the probability that it is labeled incorrectly?

$$\text{Gini}(S) = 1 - \sum_{i \in S} p_i^2$$

### Information Gain with Gini Index

$$\text{IG}(S, C) = \text{Gini}(S) - \sum_{C_i \in C} \frac{|C_i|}{|S|} \text{Gini}(C_i)$$

## Regression Trees

### Splitting Continuous Values

Class discussion: how would you do it?

In [ ]:
# load answers after class discussion
with open('answers/regtree.txt') as f:
    for line in f:
        print line

### Impurity for regression trees
Use the _variance_ of values within a node.

$$\text{Var}(S) = \sum_{y \in S} (y - \bar{y})^2$$
$$\bar{y} = \text{Mean}(S) = \frac{1}{N_S}\sum_{y \in S} y $$

Then the information gain is
$$\text{IG}(S, C) = Var(S) - \sum_{C_i \in C} \frac{|C_i|}{|S|} Var(C_i)$$

## Predicting

The final predictions of a decision tree and a regression tree are implemented differently:

### Classification

* Majority class in leaf node
* Better to predict probability of label (impure leaf nodes)

### Regression

* Mean of values in leaf node
* Linear Regression (ooo, fancy)
    * Also called *model trees*

## Recursion

Recursion uses the idea of "divide and conquer" to solve problems. It divides a complex problem you are working on into smaller sub-problems that are easily solved, rather than trying to solve the complex problem directly.

Recursive functions split the problem into two cases: the ***base case*** and the ***recursive case***. The function continually loops, calling itself, until it reaches the base case.

* Base case: Stopping criteria, the simplest case that can be solved directly.
* Recursive case: Function that splits the problem into the smaller subproblems.
    
### Three Laws of Recursion

1. A recursive algorithm must have a base case.
2. A recursive algorithm must change its state and move toward the base case.
3. A recursive algorithm must call itself, recursively.
    
### Example: Factorial

Are the following functions the same?

$$ f(x) = \prod_{i=1}^xi $$

$$f(x) =
\left\{
	\begin{array}{ll}
		1  & \mbox{if } x \leq 1 \\
		xf(x-1) & \mbox otherwise
	\end{array}
\right.
$$

In [ ]:
def factorial(x):
    """Recursively calculate x!"""
    # base case is when we get to x=0, which is 0! = 1
    if x <= 1:
        return 1
    # otherwise, recursive case, notice how we are reducing x
    else:
        return x * factorial(x-1)

In [ ]:
factorial(4)

***Map out on whiteboard***

In [ ]:
factorial(25)

In [ ]:
def power(base, exp):
    """Recursively caclulate base ** exp"""
    # base case is when exp = 0, base ** 0 = 1
    if exp <= 0:
        return 1
    #  otherwise, recursive case, reduce exp
    return base * power(base, exp - 1)

In [ ]:
power(3, 4)

***Map out on whiteboard***

In [ ]:
%%timeit -n 20
power(25, 5)

In [ ]:
%%timeit -n 20
25 ** 5

Recursion not always the answer, but useful tool to have.

### You do, fill in the following function

In [ ]:
def len_str(s):
    """Recursively determine the length of a string"""
    # base case, when should you stop?  
    # recursive case, how can you reduce your problem?
    pass

In [ ]:
#test your function

## DT Pseudocode

Recursive partitioning algorithm

```
function BuildTree:
    # base case, stop building tree
    If every item in the dataset is in the same class
    or there is not feature left to split the data:
        return a leaf node with the class label
    # recursive case, keep splitting stuff
    Else:
        find the best feature and value to split the data
        split the dataset into two subsets
        create a node
        for each subset
            call Build Tree and add the result as a child of the node
        return node
```

## Performance of Decision Trees

Class discussion: what are some problems you forsee with this approach?

http://www.r2d3.us/visual-intro-to-machine-learning-part-1/

Let's go back to the dogs and horses.

In [ ]:
def plot_classification_tree(ax, X, y, model=None):
    """Plot the horse dog data"""
    ax.plot(X[y==0, 0], X[y==0, 1], 'r.', label='Horse')
    ax.plot(X[y==1, 0], X[y==1, 1], 'b.', label='Dog')
    ax.set_title("Classifying Horses and Dogs with Decision Trees")
    ax.set_xlabel("Weight (lbs)")
    ax.set_ylabel("Height (ft)")
    ax.legend(loc='upper left')
    
    if model is None:
        model = tree.DecisionTreeClassifier()
    model.fit(X, y)
    # preserve upper limits
    xlim = (0, ax.get_xlim()[1])
    ylim = (0, ax.get_ylim()[1])
    plot_classification_thresholds(ax, X, y, 0, model.tree_, xlim, ylim)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

def plot_classification_thresholds(ax, X, y, inode, tree, xlim, ylim):
    threshold = tree.threshold[inode]
    if tree.feature[inode] == -2:
        color = ['r', 'b'][np.argmax(tree.value[inode])]
        ax.add_patch(Rectangle((xlim[0], ylim[0]), xlim[1]-xlim[0], ylim[1]-ylim[0], fc=color, alpha=0.3))
        return
    if tree.feature[inode] == 0:
        ax.plot((threshold, threshold), ylim, 'k', lw=0.5)
        plot_classification_thresholds(ax, X, y, tree.children_left[inode],  tree, (xlim[0], threshold), ylim)
        plot_classification_thresholds(ax, X, y, tree.children_right[inode], tree, (threshold, xlim[1]), ylim)
    else:
        ax.plot(xlim, (threshold, threshold), 'k', lw=0.5)
        plot_classification_thresholds(ax, X, y, tree.children_left[inode],  tree, xlim, (ylim[0], threshold))
        plot_classification_thresholds(ax, X, y, tree.children_right[inode], tree, xlim, (threshold, ylim[1]))        


In [ ]:
X, y = get_new_ugly_sample()

In [ ]:
model = tree.DecisionTreeClassifier()
fig, ax = plt.subplots(figsize=(12,8))
plot_classification_tree(ax, X, y, model)

Clearly there is some risk of overfitting.

## Picking the Best Tree

* Cross validate!
    * Train trees with different parameters, see which performs best on validation set
    * No different than any other model

## Pruning

We call the idea of modifying a decision tree to improve its performance ***pruning***.

There are two approaches: ***pre-pruning*** and ***post-pruning***.

### Pre

Prune as we build the tree, control hyper-parameters

Class discussion: what can we control?

In [ ]:
with open('answers/preprune.txt') as f:
    for line in f:
        print line

### Post

Prune after we build the complete tree

* Merge leaves if doing so decreases test-set error

#### Pseudocode
```
function Prune:
    if either left or right is not a leaf:
        call Prune on those that aren't
    if both left and right are (now) leaf nodes:
        calculate error associated with merging two leaf nodes
        calculate error associated without merging two leaf nodes
        if merging results in lower error:
            merge the leaf nodes
```

## Pros and Cons of Decision Trees

Class discussion: what do you think?

In [ ]:
# load answers after class discussion
with open('answers/pc_dt.txt') as f:
    for line in f:
        print line

## The Decision Tree Algorithms

There are some famous variants of the decision tree algorithm:

### ID3

Short for Iterative Dichotomiser 3, the original Decision Tree algorithm developed by Ross Quinlan (who's responsible for a lot of proprietary decision tree algorithms) in the 1980's.

* designed for only categorial features
* splits categorical features completely
* uses entropy and information gain to pick the best split

### CART

Short for Classification and Regression Tree was invented about the same time as ID3 by Breiman, Friedman, Olshen and Stone. The CART algorithm has the following properties:

* handles both categorial and continuous data
* always uses binary splits
* uses gini impurity to pick the best split

Algorithms will be called CART even if they don't follow all of the specifications of the original algorithm.

### C4.5

This is Quinlan's first improvement on the ID3 algorithm. The main improvements are:

* handles continuous data
* implements pruning to reduce overfitting

### C5.0

This is supposedly better, but it's proprietary so we don't have access to the specifics of the improvements.

## Objectives [afternoon-DT]
Reinforces the following objectives

* **Describe** pros/cons of decision tree algorithm
* **Implement** pseudocode for decision tree algorithm
    * Writing code that implements CART algorithm
* **Describe** common measures for making splits in a decision tree
    * Classification
        * Writing code to calculate entropy and Gini
* **Demonstrate** concept of recursion and **relate** it to decision trees
    * * Writing code that implements CART algorithm
    
#### Extra Credit 1
* **State** pruning techniques and pros/cons of each
    * Pre-pruning
        * Writing code to implement pre-pruning
    * Post-pruning
        * Writing code to implement post-pruning
        
#### Extra Credit 2
* **Describe** common measures for making splits in a decision tree
    * Regression
        * Writing code to implement regression tree